In [62]:
import pandas as pd
import numpy as np

In [95]:
prompt_types = ["yes_or_no", "no_or_yes", "agreement", "agreement_negation", "disagreement", "disagreement_negation", "options", "options_flipped"]

In [96]:
results = pd.read_csv("../runs_05_14/meta-llama/Llama-3.2-1B-results.csv")

In [97]:
def convert_version_to_target(prompt_type, version):
    match prompt_type:
        case "yes_or_no" | "no_or_yes":
            match version:
                case "unambiguous_covered": 
                    return "Yes"
                case "unambiguous_uncovered":
                    return "No"
                case _:
                    return None
        case "agreement"|"disagreement_negation":
            match version:
                case "unambiguous_covered": 
                    return "Yes"
                case "unambiguous_uncovered":
                    return "No"
                case _:
                    return None
        case "agreement_negation"|"disagreement": 
            match version:
                case "unambiguous_covered": 
                    return "No"
                case "unambiguous_uncovered":
                    return "Yes"
                case _:
                    return None

        case "options":
            match version:
                case "unambiguous_covered": 
                    return "A"
                case "unambiguous_uncovered":
                    return "B"
                case _:
                    return None

        case "options_flipped":
            match version:
                case "unambiguous_covered": 
                    return "B"
                case "unambiguous_uncovered":
                    return "A"
                case _:
                    return None        
        case _:
            return None
    

In [98]:
results["target"] = results[["prompt_type", "version"]].apply(
    lambda x: convert_version_to_target(x["prompt_type"], x["version"]), axis=1)

In [99]:
np.sum(results["version"] != "controversial")

np.int64(736)

In [100]:
results_non_controversial = results[results["version"] != "controversial"]

In [112]:
def prompt_type_correct_incorrect(prompt_type, results_non_controversial):
    pt_results = results_non_controversial[results_non_controversial["prompt_type"] == prompt_type]
    return np.sum(pt_results["target"] == pt_results["output"]), np.sum(pt_results["target"] != pt_results["output"]), pt_results.output.value_counts()
    

In [116]:
for prompt_type in prompt_types:
    n_correct, n_incorrect, value_counts = prompt_type_correct_incorrect(prompt_type, results_non_controversial)
    print(prompt_type, n_correct,  n_incorrect, value_counts)

yes_or_no 35 57 output
Yes     69
Yes!    23
Name: count, dtype: int64
no_or_yes 45 47 output
Yes         90
Penelope     2
Name: count, dtype: int64
agreement 46 46 output
Yes    92
Name: count, dtype: int64
agreement_negation 46 46 output
Yes    92
Name: count, dtype: int64
disagreement 46 46 output
Yes    92
Name: count, dtype: int64
disagreement_negation 46 46 output
Yes    92
Name: count, dtype: int64
options 45 47 output
A         90
Aneesh     2
Name: count, dtype: int64
options_flipped 21 71 output
C    51
A    41
Name: count, dtype: int64


In [110]:
results_non_controversial.output.value_counts()

output
Yes         527
A           131
C            51
Yes!         23
Penelope      2
Aneesh        2
Name: count, dtype: int64

In [111]:
results_controversial = results[results["version"] == "controversial"]

In [117]:
for prompt_type in prompt_types:
    n_correct, n_incorrect, value_counts = prompt_type_correct_incorrect(prompt_type, results_controversial)
    print(prompt_type, n_correct,  n_incorrect, value_counts)

yes_or_no 0 46 output
Yes     34
Yes!    12
Name: count, dtype: int64
no_or_yes 0 46 output
Yes         45
Penelope     1
Name: count, dtype: int64
agreement 0 46 output
Yes    46
Name: count, dtype: int64
agreement_negation 0 46 output
Yes    46
Name: count, dtype: int64
disagreement 0 46 output
Yes    46
Name: count, dtype: int64
disagreement_negation 0 46 output
Yes    46
Name: count, dtype: int64
options 0 46 output
A         45
Aneesh     1
Name: count, dtype: int64
options_flipped 0 46 output
A    24
C    22
Name: count, dtype: int64
